In [1]:
# Import all libraries needed for the tutorial

# General syntax to import specific functions in a library: 
##from (library) import (specific library function)
from pandas import DataFrame, read_csv

# General syntax to import a library but no functions: 
##import (library) as (give the library a nickname/alias)
import matplotlib.pyplot as plt
import pandas as pd #this is how I usually import pandas
import sys #only needed to determine Python version number
import matplotlib #only needed to determine Matplotlib version number

# Enable inline plotting
%matplotlib inline

import scipy
import numpy as np
#import scipy.signal as signal
from scipy.signal import *
import matplotlib.pyplot as plt
from AdaBandFlt import *
from mpl_toolkits.mplot3d import Axes3D
from sklearn.decomposition import PCA

%matplotlib tk

In [2]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_B_tstamp.csv'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2_tstamp.csv'

# create dataframe
df_ = pd.read_csv(Location, sep=',')

In [5]:
df_.head()

,T[s],sppa0001_21_Timestamp_Unit0,T[s].1,sppa0001_31_Timestamp_Unit0,T[s].2,sppa0001_41_Timestamp_Unit0,T[s].3,sppa0001_22_Timestamp_Unit0,T[s].4,sppa0001_32_Timestamp_Unit0,T[s].5,sppa0001_42_Timestamp_Unit0,T[s].6,sppa0001_23_Timestamp_Unit0,T[s].7,sppa0001_33_Timestamp_Unit0,T[s].8,sppa0001_43_Timestamp_Unit0,T[s].9,sppa0001_15_Timestamp_Unit0
0,1.63776,NaN,0.38532,NaN,0.85184,NaN,2.14396,NaN,2.13620,NaN,0.56064,NaN,2.14936,NaN,0.47808,NaN,2.13872,NaN,7.96416,NaN
1,1.99384,NaN,1.08808,NaN,2.12424,NaN,2.15492,NaN,2.14376,NaN,2.14380,NaN,2.15504,NaN,0.90124,NaN,2.14168,NaN,16.50048,NaN
2,2.13476,NaN,2.11608,NaN,2.13104,NaN,2.16148,NaN,2.15196,NaN,2.15204,NaN,2.16224,NaN,2.11860,NaN,2.14432,NaN,17.73908,NaN
3,2.13824,NaN,2.12060,NaN,2.13352,NaN,2.16732,NaN,2.15752,NaN,2.15772,NaN,2.17080,NaN,2.14616,NaN,2.15276,NaN,NaN,NaN
4,2.14420,NaN,2.13140,NaN,2.13628,NaN,2.17520,NaN,2.16072,NaN,2.16072,NaN,2.17296,NaN,2.14848,NaN,2.15540,NaN,NaN,NaN


In [22]:
data = df_.iloc[:, 2].dropna()

centers = []
for elm in data:
    centers.append(int(elm*25000))
    
len(centers)

465

In [7]:
#load data
# file path of csv file
#Location = r'/Users/33614/ExternalDrive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/SYL21/D_Drive/SUPAERO/PIR_2A/Data/data_spikes/E18KABaseline_Bcut.txt'
#Location = r'/Users/louiseplacidet/Desktop/PIR/Data/data_spikes/E18KABaseline_Bcut.txt'
Location = r'/Users/louiseplacidet/Desktop/PIR/Data/new_spike_data/newdata/E18KABaseline_BcutV2groundAll.txt'

# create dataframe
df = pd.read_csv(Location, sep='\t',skiprows=[0,1,3] , index_col='%t           ')

In [8]:
#####################################################################################################################
####  BANK OF PARTS OF DATA

all_raw_data = df #Entire recording from all electrodes

full_signal = df.iloc[:,1] #Entire recording from the first electrode

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = filtfilt(b, a, data)
    return y

# Sample rate and desired cutoff frequencies (in Hz).
fs = 25000.0
lowcut = 100.0
highcut = 2500.0


y = butter_bandpass_filter(df.iloc[:,1], lowcut, highcut, fs, order=6)


filtereddf = pd.DataFrame(y)
filtereddf.index = df.index

signal_filtered = filtereddf.iloc[:,0] #Entire recording filtered by bandpass, for one electrode


fs = 25000

xminnoise = int(np.round(11114*(fs/1000)))
xmaxnoise = int(np.round(18511*(fs/1000)))

noise_data = filtereddf.iloc[xminnoise:xmaxnoise,0]

xminspike = int(np.round(130826*(fs/1000)))
xmaxspike = int(np.round(131699*(fs/1000)))

burst_data = filtereddf.iloc[xminspike:xmaxspike,0]

In [13]:
#record spike

def record_spikes(signal, fs, spike_centers,
                  t_before = 0.001,
                  t_after = 0.002):
    
    data = np.array([[float(x) for x in range(int(np.round(fs*(t_before+t_after)))+1)]])
    
    initial_index = int(np.round(signal.index[0]*fs/1000))
    print('initial index:'+str(initial_index))
    
    for center in spike_centers:
        if center < int(np.round(fs*t_before)) + initial_index:
            spike = [0 for i in range(0, int(np.round(fs*t_before-(center-initial_index))))]
            #print('spike before concat:'+ str(len(spike)))
            #spike = np.concatenate(spike, signal.iloc[:center + int(np.round(fs*t_after))].values())
            spike = np.concatenate((spike, signal.values[:center + int(np.round(fs*t_after)) - initial_index]))
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        elif center > int(np.round(len(signal)-fs*t_after)) + initial_index:
            #spike = signal.iloc[int(np.round(center-fs*t_before)):].values()
            spike = signal.values[int(np.round(center-fs*t_before))-initial_index:]
            #print('spike before concat:'+str(len(spike)))
            spike = np.concatenate((spike,[0 for i in range(0, int(np.round(fs*t_after-(len(signal)+initial_index-center))))]))
            #print('spike after concat:'+str(len(spike)))
            data = np.insert(data, len(data), spike, axis=0)
            
        else :
            #spike = signal.iloc[int(np.round(center - fs*t_before)): int(np.round(center + fs*t_after))+1].values()
            spike = signal.values[int(np.round(center - fs*t_before))-initial_index: int(np.round(center + fs*t_after))+1-initial_index]
            data = np.insert(data, len(data), spike, axis=0)

    print(np.shape(data))
    data = data.transpose()
    #print(data)
    spike_data = pd.DataFrame(data)
    
    return spike_data

In [28]:
spike_data = record_spikes(signal_filtered, fs, centers, 
                  t_before = 0.001, 
                  t_after = 0.001)

spike_data

initial index:0
(466, 51)


,0,1,2,3,4,5,6,7,8,9,...,456,457,458,459,460,461,462,463,464,465
0,0.0,1.363464,-0.578875,1.233657,0.885997,5.640350,3.112715,10.987722,8.052379,-3.724472,...,-0.421687,-0.298185,13.387408,-1.812132,-10.036423,-9.067635,2.238396,7.440876,-3.071501,1.013984
1,1.0,1.610064,-0.315449,0.442971,1.755585,5.272174,2.880026,11.556038,9.618283,-3.203024,...,0.602600,-1.482985,15.845345,-2.171991,-10.127306,-7.369729,4.064367,6.434953,-1.605611,0.764516
2,2.0,1.416792,-0.011409,0.078931,2.414618,5.346127,2.488605,11.946987,10.110730,-2.423531,...,1.752872,-1.822712,17.991049,-2.050997,-9.404248,-5.325253,4.994243,4.960559,0.339278,0.543157
3,3.0,0.969553,0.333972,0.271105,2.822554,5.826080,2.032702,12.289381,9.301293,-1.133466,...,2.855655,-1.400076,19.665377,-1.329584,-7.852688,-3.221793,4.975656,3.268286,2.385159,0.554958
4,4.0,0.509684,0.723239,1.012134,3.060160,6.499561,1.569555,12.629931,7.175968,0.867547,...,3.701026,-0.457441,20.737212,0.086501,-5.611300,-1.325504,4.184074,1.673680,4.157726,0.977323
5,5.0,0.226386,1.144118,2.150595,3.267223,7.041028,1.106551,12.913564,4.005443,3.644467,...,4.115063,0.696465,21.112636,2.226934,-2.944815,0.179378,2.980799,0.501368,5.375549,1.885363
6,6.0,0.183792,1.558927,3.424187,3.562938,7.123206,0.605242,13.011133,0.326996,7.077670,...,4.024056,1.790660,20.748870,5.016571,-0.173184,1.233521,1.826957,0.018217,5.907800,3.207883
7,7.0,0.312147,1.910387,4.527198,3.986730,6.539578,0.004970,12.781041,-3.164427,10.868147,...,3.485118,2.670103,19.671532,8.256204,2.417256,1.916700,1.176070,0.371236,5.786688,4.734208
8,8.0,0.462788,2.145476,5.196399,4.485281,5.293663,-0.738089,12.139413,-5.784254,14.601223,...,2.671965,3.323058,17.989224,11.640671,4.648446,2.428934,1.371941,1.547517,5.175856,6.173022
9,9.0,0.497727,2.248082,5.291941,4.948269,3.620253,-1.609052,11.112029,-7.031228,17.853375,...,1.821904,3.853223,15.897020,14.815480,6.485530,3.041338,2.575822,3.369585,4.309526,7.243421


In [31]:
plt.figure(figsize = (100,20))
#plt.plot(spike_data)
spike_data.iloc[:,1:200].plot()
plt.xlabel('Time')
plt.grid()